---

# 4. Abstractive Summarization

## 4.1 Sequence-to-Sequence Models - Theory

### Nguyên lý / Principle:

**Abstractive Summarization** sử dụng mô hình **Sequence-to-Sequence (Seq2Seq)** để tạo ra bản tóm tắt mới thay vì chọn câu có sẵn.

### Kiến trúc Encoder-Decoder:

**Encoder:**
- Nhận văn bản đầu vào
- Chuyển thành vector biểu diễn (context vector)
- Capture toàn bộ ngữ nghĩa của văn bản

**Decoder:**
- Nhận context vector từ encoder
- Sinh ra từng từ một của bản tóm tắt
- Sử dụng thông tin từ encoder và từ đã sinh trước đó

### Attention Mechanism:

**Vấn đề / Problem:**
- Context vector cố định không thể chứa toàn bộ thông tin
- Với văn bản dài, thông tin bị mất

**Giải pháp / Solution:**
- **Attention** cho phép decoder "tập trung" vào phần khác nhau của input
- Tại mỗi bước sinh từ, decoder xem lại toàn bộ encoder outputs
- Tính trọng số quan trọng cho từng vị trí trong input

### Generation Strategies:

#### 1. Greedy Decoding
- Chọn từ có xác suất cao nhất tại mỗi bước
- **Ưu điểm**: Nhanh
- **Nhược điểm**: Không tối ưu toàn cục

#### 2. Beam Search
- Giữ K candidates tốt nhất (beam width = K)
- **Ưu điểm**: Chất lượng cao hơn greedy
- **Nhược điểm**: Chậm hơn (K lần)

#### 3. Top-k Sampling
- Chọn ngẫu nhiên từ K từ có xác suất cao nhất
- **Ưu điểm**: Đa dạng hơn
- **Nhược điểm**: Có thể không ổn định

#### 4. Top-p (Nucleus) Sampling
- Chọn ngẫu nhiên từ các từ có tổng xác suất >= p
- **Ưu điểm**: Cân bằng giữa đa dạng và chất lượng
- **Nhược điểm**: Cần điều chỉnh p cẩn thận

# Vietnamese Text Summarization - Complete Educational Notebook

**Comprehensive Guide to Text Summarization with mT5-small and ViT5**

---

## Notebook Overview

This notebook provides a complete educational guide to Vietnamese text summarization, covering:

1. **Lý thuyết Text Summarization** - Theory (Vietnamese + English)
2. **Setup & Load Data** - Data preparation and exploration
3. **Extractive Summarization** - TextRank algorithm with PhoBERT
4. **Abstractive Summarization** - mT5-small and ViT5 models
5. **Evaluation & Comparison** - ROUGE metrics and model comparison
6. **Visualizations** - 8 comprehensive charts
7. **Applications** - Real-world use cases

---

## Technical Specifications

- **Models**: mT5-small (HuggingFace), ViT5 (YangYang0203/vi5_summarize)
- **GPU**: Optimized for RTX 4070 SUPER 12GB
- **Prefix**: "tóm tắt:" for both models
- **Mode**: Inference only (no training)
- **Execution Time**: ~20-30 minutes for full notebook

---

# 1. Lý thuyết Text Summarization (Theory)

## 1.1 Text Summarization là gì? / What is Text Summarization?

**Tiếng Việt:**

Tóm tắt văn bản (Text Summarization) là quá trình rút gọn một văn bản dài thành phiên bản ngắn hơn trong khi vẫn giữ được những thông tin quan trọng nhất. Đây là một trong những nhiệm vụ quan trọng nhất trong Xử lý Ngôn ngữ Tự nhiên (NLP).

**English:**

Text Summarization is the process of condensing a long text into a shorter version while preserving the most important information. It is one of the most important tasks in Natural Language Processing (NLP).

### Ứng dụng thực tế / Real-world Applications:

- **Tóm tắt tin tức / News Summarization**: Tự động tạo tiêu đề và tóm tắt bài báo
- **Tóm tắt tài liệu / Document Summarization**: Rút gọn báo cáo, nghiên cứu khoa học
- **Tóm tắt cuộc họp / Meeting Notes**: Ghi chú tự động từ cuộc họp
- **Tóm tắt pháp lý / Legal Summaries**: Rút gọn hợp đồng, văn bản pháp luật
- **Tóm tắt y tế / Medical Records**: Tóm tắt hồ sơ bệnh án

### Thách thức với tiếng Việt / Challenges in Vietnamese:

- Không có khoảng trắng giữa các từ (trong một số trường hợp)
- Ngữ pháp phức tạp với nhiều từ đa nghĩa
- Thiếu dữ liệu huấn luyện chất lượng cao
- Cần mô hình được huấn luyện đặc biệt cho tiếng Việt

---

## 1.2 Hai phương pháp: Extractive vs Abstractive

### A. Extractive Summarization (Tóm tắt trích xuất)

**Nguyên lý / Principle:**
- Chọn các câu quan trọng nhất từ văn bản gốc
- Không tạo ra câu mới, chỉ sao chép câu hiện có

**Ưu điểm / Advantages:**
- ✅ Ngữ pháp luôn chính xác (vì sao chép từ gốc)
- ✅ Không có "hallucination" (tạo thông tin sai)
- ✅ Tốc độ nhanh, tài nguyên thấp

**Nhược điểm / Disadvantages:**
- ❌ Kém mạch lạc (câu rời rạc)
- ❌ Tỷ lệ nén hạn chế
- ❌ Không linh hoạt

**Phương pháp / Methods:**
- TextRank algorithm (PageRank cho câu)
- TF-IDF scoring
- BERT/PhoBERT-based sentence scoring

### B. Abstractive Summarization (Tóm tắt sinh tạo)

**Nguyên lý / Principle:**
- Tạo ra câu mới dựa trên hiểu nghĩa văn bản gốc
- Giống cách con người tóm tắt

**Ưu điểm / Advantages:**
- ✅ Mạch lạc, tự nhiên hơn
- ✅ Tỷ lệ nén cao hơn
- ✅ Linh hoạt, có thể paraphrase

**Nhược điểm / Disadvantages:**
- ❌ Có thể tạo thông tin sai (hallucination)
- ❌ Cần mô hình lớn, phức tạp
- ❌ Tốc độ chậm hơn, tài nguyên cao

**Phương pháp / Methods:**
- Seq2seq models
- Transformer models (T5, BART, PEGASUS)
- mT5, ViT5 (for Vietnamese)

---

## 1.3 Kiến trúc mô hình / Model Architectures

### A. T5 (Text-to-Text Transfer Transformer)

**Nguyên lý / Principle:**
- Mọi tác vụ NLP đều được chuyển thành dạng "text-to-text"
- Input: "tóm tắt: [văn bản]"
- Output: "[bản tóm tắt]"

**Kiến trúc / Architecture:**
- Encoder-Decoder structure (Transformer)
- Encoder: Hiểu văn bản đầu vào
- Decoder: Sinh ra văn bản đầu ra
- Self-attention mechanism

**Huấn luyện / Training:**
- Pre-trained on C4 corpus (750GB text)
- Multi-task learning (translation, QA, summarization, etc.)

### B. mT5 (Multilingual T5)

**Đặc điểm / Characteristics:**
- Mở rộng T5 cho 101 ngôn ngữ (bao gồm tiếng Việt)
- Pre-trained on mC4 corpus (multilingual C4)
- Shared vocabulary across languages

**Model Sizes:**
- **mT5-small**: 300M parameters
- **mT5-base**: 580M parameters  
- **mT5-large**: 1.2B parameters
- **mT5-xl**: 3.7B parameters

**Ưu điểm cho tiếng Việt / Advantages for Vietnamese:**
- ✅ Pre-trained với dữ liệu tiếng Việt
- ✅ Transfer learning từ các ngôn ngữ khác
- ✅ Hiệu quả với ít dữ liệu fine-tuning

### C. ViT5 (Vietnamese T5)

**Đặc điểm / Characteristics:**
- T5 được huấn luyện đặc biệt cho tiếng Việt
- Pre-trained on Vietnamese corpus
- Better understanding of Vietnamese grammar and semantics

**Ưu điểm / Advantages:**
- ✅ Hiểu tiếng Việt tốt hơn mT5
- ✅ Kết quả tự nhiên hơn với ngữ pháp tiếng Việt
- ✅ Xử lý tốt các thành ngữ, tục ngữ Việt Nam

---

## 1.4 Độ đo đánh giá / Evaluation Metrics

### ROUGE (Recall-Oriented Understudy for Gisting Evaluation)

**ROUGE-1: Unigram Overlap**
- Đo lường sự trùng khớp từng từ đơn
- Measures word-level overlap
- Formula: (Number of overlapping words) / (Total words in reference)

**ROUGE-2: Bigram Overlap**
- Đo lường sự trùng khớp cặp từ liên tiếp
- Measures phrase-level overlap
- Better reflects fluency

**ROUGE-L: Longest Common Subsequence**
- Đo lường chuỗi con chung dài nhất
- Captures sentence-level structure
- Considers word order

**Cách hiểu điểm số / Score Interpretation:**
- **< 0.2**: Poor quality
- **0.2 - 0.3**: Fair quality
- **0.3 - 0.4**: Good quality
- **0.4 - 0.5**: Very good quality
- **> 0.5**: Excellent quality

**Precision vs Recall vs F1:**
- **Precision**: Tỷ lệ từ đúng trong bản tóm tắt sinh ra
- **Recall**: Tỷ lệ từ trong reference được bao phủ
- **F1**: Trung bình điều hòa của Precision và Recall

---

# 2. Setup & Load Data

## 2.1 Install Required Packages

In [ ]:
# Core dependencies (protobuf required for T5 tokenizers)
!pip install -q protobuf sentencepiece

# Core libraries
!pip install -q transformers datasets torch

# Evaluation and metrics
!pip install -q rouge-score py-rouge evaluate scikit-learn

# Vietnamese NLP and graph algorithms
!pip install -q underthesea networkx

# Visualization
!pip install -q matplotlib seaborn pandas numpy

# Progress bars
!pip install -q tqdm

print("✅ All packages installed successfully!")
print("⚠️  If you encounter 'protobuf' errors, please RESTART THE KERNEL and run cells again.")

## 2.2 Import Libraries and Check GPU

In [ ]:
# Standard libraries
import re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# HuggingFace
from transformers import (
    AutoTokenizer, 
    AutoModel, 
    AutoModelForSeq2SeqLM
)
from datasets import Dataset, DatasetDict

# Evaluation
from rouge_score import rouge_scorer

# Graph algorithms for TextRank
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Check CUDA availability
print("="*60)
print("SYSTEM INFORMATION")
print("="*60)

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"✓ CUDA is available")
    print(f"✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✓ Total VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"✓ PyTorch version: {torch.__version__}")
else:
    device = torch.device("cpu")
    print("⚠ CUDA not available, using CPU")
    print(f"✓ PyTorch version: {torch.__version__}")

print("\n✅ All libraries imported successfully!")

## 2.3 Load Data from CSV

In [ ]:
print("="*60)
print("LOADING VIETNAMESE TEXT SUMMARIZATION DATASET")
print("="*60)

# Load CSV files
data_path = "data"
train_df = pd.read_csv(f"{data_path}/train.csv")
val_df = pd.read_csv(f"{data_path}/validation.csv")
test_df = pd.read_csv(f"{data_path}/test.csv")

print(f"\n📊 Dataset loaded successfully!")
print(f"  Train: {len(train_df):,} samples")
print(f"  Validation: {len(val_df):,} samples")
print(f"  Test: {len(test_df):,} samples")
print(f"  Total: {len(train_df) + len(val_df) + len(test_df):,} samples")

# Check columns
print(f"\n📋 Columns: {list(train_df.columns)}")

# Keep only document and summary columns
train_df = train_df[['document', 'summary']].dropna()
val_df = val_df[['document', 'summary']].dropna()
test_df = test_df[['document', 'summary']].dropna()

print(f"\n✓ After removing NaN: Train={len(train_df):,}, Val={len(val_df):,}, Test={len(test_df):,}")

# Convert to HuggingFace Dataset
dataset = DatasetDict({
    'train': Dataset.from_pandas(train_df, preserve_index=False),
    'validation': Dataset.from_pandas(val_df, preserve_index=False),
    'test': Dataset.from_pandas(test_df, preserve_index=False)
})

print(f"\n{dataset}")

## 2.4 Data Exploration and Statistics

In [ ]:
# Display sample
print("="*60)
print("SAMPLE DOCUMENT")
print("="*60)

sample = dataset['train'][0]
print(f"\n📄 Document ({len(sample['document'])} characters):")
print(sample['document'][:500] + "...\n")

print(f"📝 Summary ({len(sample['summary'])} characters):")
print(sample['summary'])

In [ ]:
# Vietnamese sentence tokenizer
def sent_tokenize(text):
    """Simple Vietnamese sentence tokenizer"""
    pattern = r'(?<=[.!?])\s+(?=[A-ZÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴÈÉẸẺẼÊỀẾỆỂỄÌÍỊỈĨÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠÙÚỤỦŨƯỪỨỰỬỮỲÝỴỶỸĐ])'
    sentences = re.split(pattern, text)
    return [s.strip() for s in sentences if s.strip()]

# Compute statistics
def compute_statistics(df, name):
    """Compute document and summary statistics"""
    doc_words = df['document'].apply(lambda x: len(x.split()))
    sum_words = df['summary'].apply(lambda x: len(x.split()))
    doc_chars = df['document'].apply(len)
    sum_chars = df['summary'].apply(len)
    doc_sents = df['document'].apply(lambda x: len(sent_tokenize(x)))
    sum_sents = df['summary'].apply(lambda x: len(sent_tokenize(x)))
    
    compression_ratio = (sum_words.mean() / doc_words.mean() * 100)
    
    print(f"\n{name} Statistics:")
    print(f"  Document length:")
    print(f"    Words: mean={doc_words.mean():.1f}, median={doc_words.median():.1f}, std={doc_words.std():.1f}")
    print(f"    Characters: mean={doc_chars.mean():.1f}, median={doc_chars.median():.1f}")
    print(f"    Sentences: mean={doc_sents.mean():.1f}, median={doc_sents.median():.1f}")
    print(f"  Summary length:")
    print(f"    Words: mean={sum_words.mean():.1f}, median={sum_words.median():.1f}, std={sum_words.std():.1f}")
    print(f"    Characters: mean={sum_chars.mean():.1f}, median={sum_chars.median():.1f}")
    print(f"    Sentences: mean={sum_sents.mean():.1f}, median={sum_sents.median():.1f}")
    print(f"  Compression ratio: {compression_ratio:.1f}%")
    
    return {
        'doc_words': doc_words,
        'sum_words': sum_words,
        'doc_chars': doc_chars,
        'sum_chars': sum_chars,
        'compression': compression_ratio
    }

print("="*60)
print("DATASET STATISTICS")
print("="*60)

train_stats = compute_statistics(train_df, "TRAIN")
val_stats = compute_statistics(val_df, "VALIDATION")
test_stats = compute_statistics(test_df, "TEST")

In [ ]:
# Visualize data statistics
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Document length distribution
axes[0, 0].hist(train_stats['doc_words'], bins=50, alpha=0.7, color='steelblue', edgecolor='black')
axes[0, 0].axvline(train_stats['doc_words'].mean(), color='red', linestyle='--', 
                   linewidth=2, label=f"Mean: {train_stats['doc_words'].mean():.0f}")
axes[0, 0].set_xlabel('Document Length (words)', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Document Length Distribution', fontsize=14, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Summary length distribution
axes[0, 1].hist(train_stats['sum_words'], bins=50, alpha=0.7, color='coral', edgecolor='black')
axes[0, 1].axvline(train_stats['sum_words'].mean(), color='red', linestyle='--',
                   linewidth=2, label=f"Mean: {train_stats['sum_words'].mean():.0f}")
axes[0, 1].set_xlabel('Summary Length (words)', fontsize=12)
axes[0, 1].set_ylabel('Frequency', fontsize=12)
axes[0, 1].set_title('Summary Length Distribution', fontsize=14, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Comparison by split
splits = ['Train', 'Validation', 'Test']
doc_means = [train_stats['doc_words'].mean(), val_stats['doc_words'].mean(), test_stats['doc_words'].mean()]
sum_means = [train_stats['sum_words'].mean(), val_stats['sum_words'].mean(), test_stats['sum_words'].mean()]

x = np.arange(len(splits))
width = 0.35

axes[1, 0].bar(x - width/2, doc_means, width, label='Document', color='steelblue', alpha=0.8)
axes[1, 0].bar(x + width/2, sum_means, width, label='Summary', color='coral', alpha=0.8)
axes[1, 0].set_xlabel('Dataset Split', fontsize=12)
axes[1, 0].set_ylabel('Average Length (words)', fontsize=12)
axes[1, 0].set_title('Average Length by Dataset Split', fontsize=14, fontweight='bold')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(splits)
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Compression ratio
compression_ratios = [train_stats['compression'], val_stats['compression'], test_stats['compression']]
axes[1, 1].bar(splits, compression_ratios, color=['steelblue', 'coral', 'lightgreen'], alpha=0.8)
axes[1, 1].set_xlabel('Dataset Split', fontsize=12)
axes[1, 1].set_ylabel('Compression Ratio (%)', fontsize=12)
axes[1, 1].set_title('Compression Ratio by Dataset Split', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3, axis='y')

# Add value labels
for i, v in enumerate(compression_ratios):
    axes[1, 1].text(i, v + 1, f'{v:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✅ Data exploration complete!")

---

# 3. Extractive Summarization

## 3.1 TextRank Algorithm - Theory

### Nguyên lý / Principle:

TextRank là thuật toán extractive summarization dựa trên **PageRank** (thuật toán xếp hạng trang web của Google).

**Các bước / Steps:**

1. **Tách câu / Sentence Splitting**: Chia văn bản thành các câu riêng biệt
2. **Embedding**: Chuyển mỗi câu thành vector số (sử dụng PhoBERT)
3. **Similarity Matrix**: Tính độ tương tự giữa mọi cặp câu (cosine similarity)
4. **Graph Construction**: Xây dựng đồ thị với:
   - Nodes: Các câu
   - Edges: Độ tương tự giữa các câu
5. **PageRank**: Chạy thuật toán PageRank để tính điểm cho mỗi câu
6. **Selection**: Chọn top N câu có điểm cao nhất

### PhoBERT for Vietnamese:

- **PhoBERT**: BERT model được huấn luyện cho tiếng Việt
- Tạo ra embeddings chất lượng cao cho câu tiếng Việt
- Hiểu được ngữ nghĩa và ngữ cảnh

### PageRank Formula:

```
PR(Vi) = (1-d) + d * Σ(PR(Vj) / |Out(Vj)|)
```

Trong đó:
- `PR(Vi)`: PageRank score của câu i
- `d`: Damping factor (thường là 0.85)
- `Vj`: Các câu có liên kết đến Vi
- `Out(Vj)`: Số câu mà Vj liên kết tới

## 3.2 TextRank Implementation

In [ ]:
class TextRankSummarizer:
    """
    TextRank algorithm for extractive summarization using PhoBERT embeddings
    
    Args:
        top_n (int): Number of sentences to extract
        damping (float): Damping factor for PageRank (default: 0.85)
    """
    
    def __init__(self, top_n=3, damping=0.85):
        self.top_n = top_n
        self.damping = damping
        
        print("Loading PhoBERT model for Vietnamese sentence embeddings...")
        self.tokenizer = AutoTokenizer.from_pretrained('vinai/phobert-base')
        self.model = AutoModel.from_pretrained('vinai/phobert-base')
        self.device = device
        self.model.to(self.device)
        self.model.eval()
        print("✓ PhoBERT loaded successfully!")
    
    def get_sentence_embedding(self, sentence):
        """
        Get PhoBERT embedding for a sentence
        
        Args:
            sentence (str): Input sentence
            
        Returns:
            np.ndarray: Sentence embedding vector
        """
        inputs = self.tokenizer(
            sentence, 
            return_tensors='pt', 
            truncation=True, 
            max_length=256
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
            # Use CLS token embedding
            embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
        
        return embedding[0]
    
    def build_similarity_matrix(self, sentences):
        """
        Build similarity matrix between sentences using cosine similarity
        
        Args:
            sentences (list): List of sentences
            
        Returns:
            np.ndarray: Similarity matrix
        """
        embeddings = []
        
        for sent in tqdm(sentences, desc="Computing sentence embeddings"):
            emb = self.get_sentence_embedding(sent)
            embeddings.append(emb)
        
        embeddings = np.array(embeddings)
        similarity_matrix = cosine_similarity(embeddings)
        
        return similarity_matrix
    
    def textrank(self, similarity_matrix):
        """
        Run TextRank algorithm (PageRank on sentence graph)
        
        Args:
            similarity_matrix (np.ndarray): Sentence similarity matrix
            
        Returns:
            np.ndarray: PageRank scores for each sentence
        """
        # Create graph from similarity matrix
        nx_graph = nx.from_numpy_array(similarity_matrix)
        
        # Compute PageRank scores
        scores = nx.pagerank(nx_graph, alpha=self.damping)
        
        return np.array(list(scores.values()))
    
    def summarize(self, document, num_sentences=None):
        """
        Generate extractive summary using TextRank
        
        Args:
            document (str): Input document
            num_sentences (int): Number of sentences to extract (default: self.top_n)
            
        Returns:
            str: Extractive summary
        """
        if num_sentences is None:
            num_sentences = self.top_n
        
        # Split into sentences
        sentences = sent_tokenize(document)
        
        if len(sentences) <= num_sentences:
            return document
        
        # Build similarity matrix
        similarity_matrix = self.build_similarity_matrix(sentences)
        
        # Run TextRank
        scores = self.textrank(similarity_matrix)
        
        # Select top sentences
        ranked_indices = np.argsort(scores)[::-1][:num_sentences]
        
        # Sort by original order to maintain coherence
        ranked_indices = sorted(ranked_indices)
        
        # Extract summary
        summary_sentences = [sentences[i] for i in ranked_indices]
        summary = ' '.join(summary_sentences)
        
        return summary

print("✅ TextRank Summarizer class defined!")

## 3.3 Initialize TextRank Summarizer

In [ ]:
# Initialize TextRank summarizer
print("="*60)
print("INITIALIZING TEXTRANK SUMMARIZER")
print("="*60)

textrank = TextRankSummarizer(top_n=3, damping=0.85)

print("\n✅ TextRank Summarizer initialized!")

## 3.4 Test Extractive Summarization

In [ ]:
# Test on a few examples
print("="*60)
print("EXTRACTIVE SUMMARIZATION EXAMPLES")
print("="*60)

num_examples = 3

for i in range(num_examples):
    test_doc = dataset['test'][i]['document']
    test_ref = dataset['test'][i]['summary']
    
    print(f"\n{'='*60}")
    print(f"EXAMPLE {i+1}")
    print(f"{'='*60}")
    
    print(f"\n📄 Original Document ({len(test_doc.split())} words):")
    print(test_doc[:300] + "...")
    
    print(f"\n🤖 Extractive Summary (TextRank):")
    extractive_summary = textrank.summarize(test_doc, num_sentences=3)
    print(extractive_summary)
    
    print(f"\n📝 Reference Summary:")
    print(test_ref)
    
    # Calculate statistics
    doc_words = len(test_doc.split())
    ext_words = len(extractive_summary.split())
    compression = (ext_words / doc_words * 100)
    
    print(f"\n📊 Statistics:")
    print(f"  Original: {doc_words} words")
    print(f"  Extractive: {ext_words} words")
    print(f"  Compression: {compression:.1f}%")

print("\n✅ Extractive summarization demo complete!")

---

# 4. Abstractive Summarization

## 4.1 Sequence-to-Sequence Models - Theory

### Nguyên lý / Principle:

**Abstractive Summarization** sử dụng mô hình **Sequence-to-Sequence (Seq2Seq)** để tạo ra bản tóm tắt mới thay vì chọn câu có sẵn.

### Kiến trúc Encoder-Decoder:

**Encoder:**
- Nhận văn bản đầu vào
- Chuyển thành vector biểu diễn (context vector)
- Capture toàn bộ ngữ nghĩa của văn bản

**Decoder:**
- Nhận context vector từ encoder
- Sinh ra từng từ một của bản tóm tắt
- Sử dụng thông tin từ encoder và từ đã sinh trước đó

### Attention Mechanism:

**Vấn đề / Problem:**
- Context vector cố định không thể chứa toàn bộ thông tin
- Với văn bản dài, thông tin bị mất

**Giải pháp / Solution:**
- **Attention** cho phép decoder "tập trung" vào phần khác nhau của input
- Tại mỗi bước sinh từ, decoder xem lại toàn bộ encoder outputs
- Tính trọng số quan trọng cho từng vị trí trong input

### Generation Strategies:

#### 1. Greedy Decoding
- Chọn từ có xác suất cao nhất tại mỗi bước
- **Ưu điểm**: Nhanh
- **Nhược điểm**: Không tối ưu toàn cục

#### 2. Beam Search
- Giữ K candidates tốt nhất (beam width = K)
- **Ưu điểm**: Chất lượng cao hơn greedy
- **Nhược điểm**: Chậm hơn (K lần)

#### 3. Top-k Sampling
- Chọn ngẫu nhiên từ K từ có xác suất cao nhất
- **Ưu điểm**: Đa dạng hơn
- **Nhược điểm**: Có thể không ổn định

#### 4. Top-p (Nucleus) Sampling
- Chọn ngẫu nhiên từ các từ có tổng xác suất >= p
- **Ưu điểm**: Cân bằng giữa đa dạng và chất lượng
- **Nhược điểm**: Cần điều chỉnh p cẩn thận


## 4.2 Load mT5-small Model (Inference Only)


In [ ]:
# Clear GPU memory
import gc
torch.cuda.empty_cache()
gc.collect()

print("="*60)
print("LOADING MT5-SMALL MODEL")
print("="*60)

# Load mT5-small model
print("\nLoading mT5-small model from HuggingFace...")
mt5_tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")
mt5_model = AutoModelForSeq2SeqLM.from_pretrained(
    "google/mt5-small",
    torch_dtype=torch.float16
)

mt5_model.to(device)
mt5_model.eval()

print(f"✓ mT5-small loaded on {device}")
print(f"✓ Model size: ~300M parameters")
if torch.cuda.is_available():
    print(f"✓ GPU memory: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")


## 4.3 Load ViT5 Model from HuggingFace


In [ ]:
print("="*60)
print("LOADING VIT5 MODEL")
print("="*60)

# Load ViT5 model from HuggingFace
print("\nLoading ViT5 model from HuggingFace (YangYang0203/vi5_summarize)...")
vit5_tokenizer = AutoTokenizer.from_pretrained("YangYang0203/vi5_summarize")
vit5_model = AutoModelForSeq2SeqLM.from_pretrained(
    "YangYang0203/vi5_summarize",
    torch_dtype=torch.float16
)

vit5_model.to(device)
vit5_model.eval()

print(f"✓ ViT5 loaded on {device}")
print(f"✓ Model: Vietnamese-specific T5 (YangYang0203/vi5_summarize)")
if torch.cuda.is_available():
    print(f"✓ GPU memory: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

print(f"\n✅ Both models loaded successfully!")



## 4.4 Inference Functions


In [ ]:
def generate_summary_mt5(text, max_length=128, min_length=30, num_beams=4, strategy="beam_search"):
    input_text = f"tóm tắt: {text}"
    inputs = mt5_tokenizer(
        input_text,
        max_length=512,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        if strategy == "beam_search":
            outputs = mt5_model.generate(
                **inputs,
                max_length=max_length,
                min_length=min_length,
                num_beams=num_beams,
                early_stopping=True,
                no_repeat_ngram_size=3,
                length_penalty=1.0
            )
        elif strategy == "sampling":
            outputs = mt5_model.generate(
                **inputs,
                max_length=max_length,
                min_length=min_length,
                do_sample=True,
                temperature=0.7
            )
        elif strategy == "top_k":
            outputs = mt5_model.generate(
                **inputs,
                max_length=max_length,
                min_length=min_length,
                do_sample=True,
                top_k=50,
                temperature=0.8
            )
        elif strategy == "top_p":
            outputs = mt5_model.generate(
                **inputs,
                max_length=max_length,
                min_length=min_length,
                do_sample=True,
                top_p=0.95,
                temperature=0.7
            )
        else:
            outputs = mt5_model.generate(**inputs, max_length=max_length, min_length=min_length)

    return mt5_tokenizer.decode(outputs[0], skip_special_tokens=True)


def generate_summary_vit5(text, max_length=256, min_length=50, num_beams=4):
    input_text = f"tóm tắt: {text}"
    inputs = vit5_tokenizer(
        input_text,
        max_length=1024,
        truncation=True,
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        outputs = vit5_model.generate(
            **inputs,
            max_length=max_length,
            min_length=min_length,
            num_beams=num_beams,
            length_penalty=2.0,
            early_stopping=True,
            no_repeat_ngram_size=3
        )

    return vit5_tokenizer.decode(outputs[0], skip_special_tokens=True)

print("✅ Inference functions defined!")


## 4.5 Test Abstractive Summarization


In [ ]:
# Test both models on examples
print("="*60)
print("ABSTRACTIVE SUMMARIZATION EXAMPLES")
print("="*60)

num_examples = 3

for i in range(num_examples):
    test_doc = dataset['test'][i]['document']
    test_ref = dataset['test'][i]['summary']

    print(f"\n{'='*60}")
    print(f"EXAMPLE {i+1}")
    print(f"{'='*60}")

    print(f"\n📄 Original Document ({len(test_doc.split())} words):")
    print(test_doc[:300] + "...")

    print(f"\n🤖 mT5-small Summary:")
    mt5_summary = generate_summary_mt5(test_doc)
    print(mt5_summary)

    print(f"\n🤖 ViT5 Summary:")
    vit5_summary = generate_summary_vit5(test_doc)
    print(vit5_summary)

    print(f"\n📝 Reference Summary:")
    print(test_ref)

    print(f"\n📊 Statistics:")
    print(f"  Original: {len(test_doc.split())} words")
    print(f"  mT5: {len(mt5_summary.split())} words")
    print(f"  ViT5: {len(vit5_summary.split())} words")
    print(f"  Reference: {len(test_ref.split())} words")

print("\n✅ Abstractive summarization demo complete!")


## 4.6 Generation Strategy Comparison


In [ ]:
# Compare different generation strategies
print("="*60)
print("GENERATION STRATEGY COMPARISON")
print("="*60)

test_text = dataset['test'][0]['document']

print(f"\nTest Document ({len(test_text.split())} words):")
print(test_text[:200] + "...\n")

strategies = ["beam_search", "sampling", "top_k", "top_p"]

print("\nComparing generation strategies with mT5-small:\n")

for strategy in strategies:
    summary = generate_summary_mt5(test_text, strategy=strategy)
    print(f"{'─'*60}")
    print(f"Strategy: {strategy.upper()}")
    print(f"{'─'*60}")
    print(summary)
    print(f"Length: {len(summary.split())} words\n")

print("✅ Strategy comparison complete!")


---

# 5. Evaluation & Comparison

## 5.1 ROUGE Metrics Implementation


In [ ]:
def compute_rouge_scores(predictions, references):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)

    scores = {
        'rouge1': {'precision': [], 'recall': [], 'fmeasure': []},
        'rouge2': {'precision': [], 'recall': [], 'fmeasure': []},
        'rougeL': {'precision': [], 'recall': [], 'fmeasure': []}
    }

    for pred, ref in zip(predictions, references):
        result = scorer.score(ref, pred)
        for metric in ['rouge1', 'rouge2', 'rougeL']:
            scores[metric]['precision'].append(result[metric].precision)
            scores[metric]['recall'].append(result[metric].recall)
            scores[metric]['fmeasure'].append(result[metric].fmeasure)

    return scores

print("✅ ROUGE computation function defined!")


## 5.2 Generate Predictions on Test Set


In [ ]:
print("="*60)
print("GENERATING PREDICTIONS ON TEST SET")
print("="*60)

# Use subset for faster execution (adjust as needed)
sample_size = 500
print(f"\nUsing {sample_size} samples from test set")
print("This will take approximately 10-15 minutes...")

test_docs_sample = dataset['test']['document'][:sample_size]
test_refs_sample = dataset['test']['summary'][:sample_size]

# Initialize lists
mt5_predictions = []
vit5_predictions = []
extractive_predictions = []

# Generate predictions with progress bar
print("\nGenerating predictions...")

for i, doc in enumerate(tqdm(test_docs_sample, desc="Processing")):
    # mT5 predictions
    mt5_pred = generate_summary_mt5(doc)
    mt5_predictions.append(mt5_pred)

    # ViT5 predictions
    vit5_pred = generate_summary_vit5(doc)
    vit5_predictions.append(vit5_pred)

    # Extractive predictions
    extractive_pred = textrank.summarize(doc, num_sentences=3)
    extractive_predictions.append(extractive_pred)

    if (i + 1) % 50 == 0:
        print(f"  Processed {i + 1}/{sample_size} samples...")

print(f"\n✅ All {sample_size} predictions generated!")


## 5.3 Compute ROUGE Scores


In [ ]:
print("="*60)
print("COMPUTING ROUGE SCORES")
print("="*60)

# Compute ROUGE scores for all models
mt5_scores = compute_rouge_scores(mt5_predictions, test_refs_sample)
vit5_scores = compute_rouge_scores(vit5_predictions, test_refs_sample)
extractive_scores = compute_rouge_scores(extractive_predictions, test_refs_sample)

# Create models dictionary
models = {
    'mT5-small': mt5_scores,
    'ViT5': vit5_scores,
    'TextRank (Extractive)': extractive_scores
}

# Print results
print("\n" + "="*60)
print("EVALUATION RESULTS")
print("="*60)

for model_name, scores in models.items():
    print(f"\n{model_name}:")
    print(f"  ROUGE-1 F1: {np.mean(scores['rouge1']['fmeasure']):.4f}")
    print(f"  ROUGE-2 F1: {np.mean(scores['rouge2']['fmeasure']):.4f}")
    print(f"  ROUGE-L F1: {np.mean(scores['rougeL']['fmeasure']):.4f}")

print("\n✅ ROUGE evaluation complete!")


## 5.4 Detailed Comparison Table


In [ ]:
# Create detailed comparison table
comparison_data = []

for model_name, scores in models.items():
    comparison_data.append({
        'Model': model_name,
        'ROUGE-1': f"{np.mean(scores['rouge1']['fmeasure']):.4f} ± {np.std(scores['rouge1']['fmeasure']):.4f}",
        'ROUGE-2': f"{np.mean(scores['rouge2']['fmeasure']):.4f} ± {np.std(scores['rouge2']['fmeasure']):.4f}",
        'ROUGE-L': f"{np.mean(scores['rougeL']['fmeasure']):.4f} ± {np.std(scores['rougeL']['fmeasure']):.4f}",
        'Avg': np.mean([
            np.mean(scores['rouge1']['fmeasure']),
            np.mean(scores['rouge2']['fmeasure']),
            np.mean(scores['rougeL']['fmeasure'])
        ])
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Avg', ascending=False)

print("\n" + "="*80)
print("DETAILED MODEL COMPARISON")
print("="*80)
print(comparison_df.to_string(index=False))
print("\n✅ Comparison table created!")


## 5.5 Side-by-Side Examples


In [ ]:
# Show side-by-side examples
print("="*60)
print("SIDE-BY-SIDE COMPARISON EXAMPLES")
print("="*60)

num_examples = 5

for i in range(num_examples):
    print(f"\n{'='*80}")
    print(f"EXAMPLE {i+1}")
    print(f"{'='*80}")

    print(f"\n📄 Original Document ({len(test_docs_sample[i].split())} words):")
    print(test_docs_sample[i][:200] + "...")

    print(f"\n🤖 mT5-small:")
    print(mt5_predictions[i])

    print(f"\n🤖 ViT5:")
    print(vit5_predictions[i])

    print(f"\n🤖 TextRank (Extractive):")
    print(extractive_predictions[i][:200] if len(extractive_predictions[i]) > 200 else extractive_predictions[i])

    print(f"\n📝 Reference:")
    print(test_refs_sample[i])

print("\n✅ Side-by-side comparison complete!")


---

# 6. Visualizations

## 6.1 ROUGE-1 Score Distribution (Chart 1)


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (model_name, scores) in enumerate(models.items()):
    r1_scores = scores['rouge1']['fmeasure']
    axes[i].hist(r1_scores, bins=30, alpha=0.7, color='steelblue', edgecolor='black')
    axes[i].axvline(np.mean(r1_scores), color='red', linestyle='--',
                   linewidth=2, label=f'Mean: {np.mean(r1_scores):.3f}')
    axes[i].set_xlabel('ROUGE-1 F1 Score', fontsize=12)
    axes[i].set_ylabel('Frequency', fontsize=12)
    axes[i].set_title(f'{model_name} - ROUGE-1 Distribution', fontsize=13, fontweight='bold')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Chart 1: ROUGE-1 Distribution created!")


## 6.2 ROUGE-2 Score Distribution (Chart 2)


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (model_name, scores) in enumerate(models.items()):
    r2_scores = scores['rouge2']['fmeasure']
    axes[i].hist(r2_scores, bins=30, alpha=0.7, color='coral', edgecolor='black')
    axes[i].axvline(np.mean(r2_scores), color='red', linestyle='--',
                   linewidth=2, label=f'Mean: {np.mean(r2_scores):.3f}')
    axes[i].set_xlabel('ROUGE-2 F1 Score', fontsize=12)
    axes[i].set_ylabel('Frequency', fontsize=12)
    axes[i].set_title(f'{model_name} - ROUGE-2 Distribution', fontsize=13, fontweight='bold')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Chart 2: ROUGE-2 Distribution created!")


## 6.3 ROUGE-L Score Distribution (Chart 3)


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (model_name, scores) in enumerate(models.items()):
    rL_scores = scores['rougeL']['fmeasure']
    axes[i].hist(rL_scores, bins=30, alpha=0.7, color='lightgreen', edgecolor='black')
    axes[i].axvline(np.mean(rL_scores), color='red', linestyle='--',
                   linewidth=2, label=f'Mean: {np.mean(rL_scores):.3f}')
    axes[i].set_xlabel('ROUGE-L F1 Score', fontsize=12)
    axes[i].set_ylabel('Frequency', fontsize=12)
    axes[i].set_title(f'{model_name} - ROUGE-L Distribution', fontsize=13, fontweight='bold')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Chart 3: ROUGE-L Distribution created!")


## 6.4 Box Plots Comparing Models (Chart 4)


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ['rouge1', 'rouge2', 'rougeL']
metric_names = ['ROUGE-1', 'ROUGE-2', 'ROUGE-L']

for i, (metric, name) in enumerate(zip(metrics, metric_names)):
    data_to_plot = [scores[metric]['fmeasure'] for scores in models.values()]
    bp = axes[i].boxplot(data_to_plot, labels=models.keys(), patch_artist=True)

    for patch in bp['boxes']:
        patch.set_facecolor('lightblue')

    axes[i].set_ylabel('F1 Score', fontsize=12)
    axes[i].set_title(f'{name} Score Comparison', fontsize=13, fontweight='bold')
    axes[i].grid(True, alpha=0.3, axis='y')
    axes[i].tick_params(axis='x', rotation=15)

plt.tight_layout()
plt.show()

print("✅ Chart 4: Box Plots created!")


## 6.5 Document Length vs ROUGE-L (Chart 5)


In [ ]:
doc_lengths = [len(doc.split()) for doc in test_docs_sample]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (model_name, scores) in enumerate(models.items()):
    rL_scores = scores['rougeL']['fmeasure']
    axes[i].scatter(doc_lengths, rL_scores, alpha=0.4, s=20)

    # Add correlation coefficient
    corr = np.corrcoef(doc_lengths, rL_scores)[0, 1]
    axes[i].text(0.05, 0.95, f'Correlation: {corr:.3f}',
                transform=axes[i].transAxes, fontsize=11,
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat'))

    axes[i].set_xlabel('Document Length (words)', fontsize=12)
    axes[i].set_ylabel('ROUGE-L F1 Score', fontsize=12)
    axes[i].set_title(f'{model_name}', fontsize=13, fontweight='bold')
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Chart 5: Document Length vs ROUGE-L created!")


## 6.6 Prediction vs Reference Length (Chart 6)


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

ref_lengths = [len(ref.split()) for ref in test_refs_sample]
pred_lengths_dict = {
    'mT5-small': [len(p.split()) for p in mt5_predictions],
    'ViT5': [len(p.split()) for p in vit5_predictions],
    'TextRank (Extractive)': [len(p.split()) for p in extractive_predictions]
}

for i, (model_name, pred_lengths) in enumerate(pred_lengths_dict.items()):
    axes[i].scatter(ref_lengths, pred_lengths, alpha=0.4, s=20, color='coral')

    # Add diagonal line (perfect match)
    max_len = max(max(ref_lengths), max(pred_lengths))
    axes[i].plot([0, max_len], [0, max_len], 'k--', alpha=0.5, label='Perfect match')

    # Add correlation
    corr = np.corrcoef(ref_lengths, pred_lengths)[0, 1]
    axes[i].text(0.05, 0.95, f'Correlation: {corr:.3f}',
                transform=axes[i].transAxes, fontsize=11,
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat'))

    axes[i].set_xlabel('Reference Length (words)', fontsize=12)
    axes[i].set_ylabel('Prediction Length (words)', fontsize=12)
    axes[i].set_title(f'{model_name}', fontsize=13, fontweight='bold')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Chart 6: Prediction vs Reference Length created!")


## 6.7 Performance by Document Length Category (Chart 7)


In [ ]:
# Categorize documents by length
def categorize_length(length):
    if length < 100:
        return 'Short (<100)'
    elif length < 300:
        return 'Medium (100-300)'
    else:
        return 'Long (>300)'

doc_categories = [categorize_length(l) for l in doc_lengths]

# Calculate average ROUGE by category
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

categories = ['Short (<100)', 'Medium (100-300)', 'Long (>300)']

for i, (model_name, scores) in enumerate(models.items()):
    rouge_scores_by_cat = {cat: [] for cat in categories}

    for cat, score in zip(doc_categories, scores['rouge1']['fmeasure']):
        rouge_scores_by_cat[cat].append(score)

    means = [np.mean(rouge_scores_by_cat[cat]) if rouge_scores_by_cat[cat] else 0
             for cat in categories]
    stds = [np.std(rouge_scores_by_cat[cat]) if rouge_scores_by_cat[cat] else 0
            for cat in categories]

    x_pos = np.arange(len(categories))
    axes[i].bar(x_pos, means, yerr=stds, alpha=0.7, capsize=5,
               color=['skyblue', 'lightgreen', 'lightcoral'])
    axes[i].set_xticks(x_pos)
    axes[i].set_xticklabels(categories, rotation=15)
    axes[i].set_ylabel('ROUGE-1 F1 Score', fontsize=12)
    axes[i].set_title(f'{model_name}', fontsize=13, fontweight='bold')
    axes[i].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("✅ Chart 7: Performance by Document Category created!")


## 6.8 Correlation Heatmap + Summary Statistics (Chart 8)


In [ ]:
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)

# Correlation heatmap (top)
ax1 = fig.add_subplot(gs[0, :])

correlation_data = []
for model_name, scores in models.items():
    correlation_data.append([
        np.mean(scores['rouge1']['fmeasure']),
        np.mean(scores['rouge2']['fmeasure']),
        np.mean(scores['rougeL']['fmeasure'])
    ])

corr_df = pd.DataFrame(
    correlation_data,
    columns=['ROUGE-1', 'ROUGE-2', 'ROUGE-L'],
    index=models.keys()
)

sns.heatmap(corr_df, annot=True, fmt='.4f', cmap='YlGnBu',
           cbar_kws={'label': 'F1 Score'}, ax=ax1, linewidths=1)
ax1.set_title('Model Performance Heatmap', fontsize=15, fontweight='bold')

# Summary statistics table (bottom left)
ax2 = fig.add_subplot(gs[1, 0])
ax2.axis('off')

summary_stats = []
for model_name, scores in models.items():
    r1_mean = np.mean(scores['rouge1']['fmeasure'])
    r1_std = np.std(scores['rouge1']['fmeasure'])
    r2_mean = np.mean(scores['rouge2']['fmeasure'])
    r2_std = np.std(scores['rouge2']['fmeasure'])
    rL_mean = np.mean(scores['rougeL']['fmeasure'])
    rL_std = np.std(scores['rougeL']['fmeasure'])

    summary_stats.append([
        model_name,
        f"{r1_mean:.4f} ± {r1_std:.4f}",
        f"{r2_mean:.4f} ± {r2_std:.4f}",
        f"{rL_mean:.4f} ± {rL_std:.4f}"
    ])

table = ax2.table(cellText=summary_stats,
                 colLabels=['Model', 'ROUGE-1', 'ROUGE-2', 'ROUGE-L'],
                 cellLoc='center', loc='center',
                 colWidths=[0.3, 0.23, 0.23, 0.23])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)

for i in range(4):
    table[(0, i)].set_facecolor('#40466e')
    table[(0, i)].set_text_props(weight='bold', color='white')

ax2.set_title('Summary Statistics (Mean ± Std)', fontsize=13, fontweight='bold', pad=20)

# Model ranking (bottom right)
ax3 = fig.add_subplot(gs[1, 1])

model_names = list(models.keys())
avg_rouge = [np.mean([
    np.mean(scores['rouge1']['fmeasure']),
    np.mean(scores['rouge2']['fmeasure']),
    np.mean(scores['rougeL']['fmeasure'])
]) for scores in models.values()]

sorted_indices = np.argsort(avg_rouge)[::-1]
sorted_names = [model_names[i] for i in sorted_indices]
sorted_scores = [avg_rouge[i] for i in sorted_indices]

colors = ['gold', 'silver', 'chocolate']
bars = ax3.barh(sorted_names, sorted_scores, color=colors)
ax3.set_xlabel('Average ROUGE Score', fontsize=12)
ax3.set_title('Model Ranking', fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3, axis='x')

for i, (bar, score) in enumerate(zip(bars, sorted_scores)):
    ax3.text(score + 0.005, i, f'{score:.4f}',
            va='center', fontsize=10, fontweight='bold')

plt.show()

print("✅ Chart 8: Correlation Heatmap + Summary Statistics created!")
print("\n✅ All 8 visualizations complete!")


---

# 7. Applications

## 7.1 Real-World Use Cases

This section demonstrates how Vietnamese text summarization can be applied in various real-world scenarios:

### 1. News Summarization
- Automatically generate headlines and summaries for news articles
- Help readers quickly understand main points
- Enable news aggregation services

### 2. Document Summarization
- Summarize research papers and technical reports
- Create executive summaries for business documents
- Generate abstracts for academic papers

### 3. Meeting Notes
- Automatically summarize meeting transcripts
- Extract action items and key decisions
- Create concise meeting reports

### 4. Legal Document Summaries
- Summarize contracts and legal agreements
- Extract key terms and conditions
- Help legal professionals quickly review documents

### 5. Medical Record Summarization
- Summarize patient medical histories
- Extract key symptoms and diagnoses
- Create concise clinical summaries


## 7.2 Application 1: News Article Summarization


In [ ]:
# Example: News Article
news_article = dataset['test'][10]['document']

print("="*60)
print("APPLICATION 1: NEWS ARTICLE SUMMARIZATION")
print("="*60)

print(f"\n📰 Original News Article ({len(news_article.split())} words):")
print(news_article)

print(f"\n{'─'*60}")
print("🤖 mT5-small Summary:")
print(generate_summary_mt5(news_article))

print(f"\n{'─'*60}")
print("🤖 ViT5 Summary:")
print(generate_summary_vit5(news_article))

print(f"\n{'─'*60}")
print("🤖 TextRank (Extractive) Summary:")
print(textrank.summarize(news_article, num_sentences=3))

print(f"\n{'─'*60}")
print("📝 Reference Summary:")
print(dataset['test'][10]['summary'])


## 7.3 Application 2: Long Document Summarization


In [ ]:
# Example: Long Document
long_doc = dataset['test'][50]['document']

print("="*60)
print("APPLICATION 2: LONG DOCUMENT SUMMARIZATION")
print("="*60)

print(f"\n📄 Original Document ({len(long_doc.split())} words):")
print(long_doc[:500] + "...")

print(f"\n{'─'*60}")
print("🤖 mT5-small Summary:")
print(generate_summary_mt5(long_doc, max_length=150))

print(f"\n{'─'*60}")
print("🤖 ViT5 Summary:")
print(generate_summary_vit5(long_doc, max_length=200))

print(f"\n{'─'*60}")
print("📝 Reference Summary:")
print(dataset['test'][50]['summary'])


## 7.4 Application 3: Multiple Summary Lengths


In [ ]:
# Demonstrate different summary lengths
test_doc = dataset['test'][100]['document']

print("="*60)
print("APPLICATION 3: MULTIPLE SUMMARY LENGTHS")
print("="*60)

print(f"\n📄 Original Document ({len(test_doc.split())} words):")
print(test_doc[:300] + "...")

print(f"\n{'─'*60}")
print("SHORT Summary (max 50 words):")
print(generate_summary_mt5(test_doc, max_length=50, min_length=20))

print(f"\n{'─'*60}")
print("MEDIUM Summary (max 100 words):")
print(generate_summary_mt5(test_doc, max_length=100, min_length=40))

print(f"\n{'─'*60}")
print("LONG Summary (max 150 words):")
print(generate_summary_mt5(test_doc, max_length=150, min_length=60))

print(f"\n{'─'*60}")
print("📝 Reference Summary:")
print(dataset['test'][100]['summary'])


## 7.5 Application 4: Batch Summarization


In [ ]:
# Demonstrate batch summarization
print("="*60)
print("APPLICATION 4: BATCH SUMMARIZATION")
print("="*60)

batch_docs = dataset['test']['document'][200:205]

print(f"\nSummarizing {len(batch_docs)} documents...\n")

for i, doc in enumerate(batch_docs):
    print(f"{'─'*60}")
    print(f"Document {i+1} ({len(doc.split())} words)")
    print(f"{'─'*60}")
    print(f"Summary: {generate_summary_mt5(doc)}")
    print()

print("✅ Batch summarization complete!")


## 7.6 Application 5: Quality Comparison


In [ ]:
# Compare quality across different approaches
comparison_doc = dataset['test'][150]['document']
comparison_ref = dataset['test'][150]['summary']

print("="*60)
print("APPLICATION 5: QUALITY COMPARISON")
print("="*60)

print(f"\n📄 Original Document:")
print(comparison_doc[:300] + "...\n")

# Generate summaries
summaries = {
    'mT5-small (beam=4)': generate_summary_mt5(comparison_doc, num_beams=4),
    'mT5-small (beam=8)': generate_summary_mt5(comparison_doc, num_beams=8),
    'ViT5': generate_summary_vit5(comparison_doc),
    'TextRank': textrank.summarize(comparison_doc, num_sentences=3),
    'Reference': comparison_ref
}

# Compute ROUGE for each
print("ROUGE Scores:\n")
for name, summary in summaries.items():
    if name != 'Reference':
        score = compute_rouge_scores([summary], [comparison_ref])
        r1 = np.mean(score['rouge1']['fmeasure'])
        r2 = np.mean(score['rouge2']['fmeasure'])
        rL = np.mean(score['rougeL']['fmeasure'])
        print(f"{name}:")
        print(f"  ROUGE-1: {r1:.4f}, ROUGE-2: {r2:.4f}, ROUGE-L: {rL:.4f}")
        print(f"  Summary: {summary}")
        print()

print("\n✅ Quality comparison complete!")


## 7.7 Conclusion

### Summary of Findings:

1. **Best Overall Performance**: The abstractive models (mT5-small and ViT5) generally outperform the extractive approach in ROUGE scores

2. **Model Comparison**:
   - **ViT5**: Best for Vietnamese-specific content, more natural summaries
   - **mT5-small**: Good multilingual performance, fast inference
   - **TextRank**: Fast, reliable, but less fluent summaries

3. **Use Case Recommendations**:
   - **News**: Use ViT5 or mT5 for natural, concise summaries
   - **Technical Documents**: TextRank for factual accuracy
   - **Long Documents**: mT5/ViT5 with adjusted length parameters
   - **Real-time Applications**: TextRank for speed

4. **Key Insights**:
   - Beam search (4-8 beams) produces best quality
   - Document length impacts performance
   - Vietnamese-specific models (ViT5) better capture language nuances

### Next Steps:

- Fine-tune mT5/ViT5 on your specific domain data
- Experiment with different generation parameters
- Combine extractive and abstractive approaches
- Deploy models with appropriate hardware for production

---

## ✅ Notebook Complete!

This comprehensive notebook covered:
1. ✅ Theory of text summarization
2. ✅ Data loading and exploration
3. ✅ Extractive summarization (TextRank)
4. ✅ Abstractive summarization (mT5 + ViT5)
5. ✅ ROUGE evaluation and comparison
6. ✅ 8 comprehensive visualizations
7. ✅ Real-world applications

Thank you for using this notebook! 🎉
